In [1]:
import math

size = int(input("Введите желаемую размерность матрицы"))

In [2]:
def get_random_matrix(size: int, max_value: int, min_value: int) -> Matrix:
    return Matrix(QQ, [[math.floor(random()*(max_value-min_value)+min_value) for i in range(size)] for _ in range(size)])
def get_random_rightside(size: int, max_value: int, min_value: int) -> vector:
    return vector([math.floor(random()*(max_value-min_value)+min_value) for i in range(size)])

In [3]:
print(get_random_matrix(size, -50, 50))
print(get_random_rightside(size, -50, 50))

[ 24  21  -7 -27  49 -49 -30]
[-34 -47 -18   4 -15 -41  29]
[-23  -1  45  26 -22  35 -14]
[ 13  46  39 -12   4  24  12]
[ 33  28  -9  33   6  25 -43]
[-10  43   2 -40 -39 -46  18]
[-24 -36   5 -11  45 -37  -2]
(23, 22, 40, 45, 32, -16, 12)


In [4]:
from typing import List, Tuple, Optional
def get_max_abs(matrix: Matrix, excluded_cols: Optional[List[int]]=None, excluded_rows: Optional[List[int]]=None) -> Tuple[int]:
    if excluded_cols is None:
        excluded_cols = []
    if excluded_rows is None:
        excluded_rows = []
    max_coordinates = (-1, -1)
    for j in range(size):
        if j in excluded_cols:
            continue
        temp_max = -1
        temp_coords = (-1, -1)
        for i in range(size):
            if i in excluded_rows:
                continue
            if abs(matrix[i, j]) > temp_max:
                temp_max = abs(matrix[i, j])
                temp_coords = (i, j)
        if max_coordinates == (-1, -1) or temp_max > abs(matrix[max_coordinates[0], max_coordinates[1]]):
            max_coordinates = temp_coords
    return max_coordinates

def nullate_elements(matrix: Matrix, rs: vector, coords: Tuple[int], excluded_rows: Optional[List[int]]=None) -> (Matrix, vector):
    rs: List[float] = rs.list()
    
    if excluded_rows is None:
        excluded_rows = []
    elem = matrix[coords[0],coords[1]]
    for i in range(size):
        matrix[coords[0],i]/=elem
    rs[coords[0]]/=elem
    for i in range(size):
        if i in excluded_rows or i == coords[0]:
            continue
        rs[i] -= matrix[i, coords[1]]*rs[coords[0]]
        coeff = matrix[i, coords[1]]
        for j in range(size):
            matrix[i, j] -= coeff*matrix[coords[0], j]
    return matrix, vector(rs)
        
        

In [5]:
# m = get_random_matrix(size, -50, 50)
# rs = get_random_rightside(size, -50, 50)
m = Matrix(QQ, [[0.411, 0.421, -0.333, 0.313, -0.141, -0.381, 0.245], [0.241, 0.705, 0.139, -0.409, 0.321, 0.0625, 0.101],
                [0.123, -0.239, 0.502, 0.901, 0.243, 0.819, 0.321], [0.413, 0.309, 0.801, 0.865, 0.423, 0.118, 0.183],
                [0.241, -0.221, -0.243, 0.134, 1.274, 0.712, 0.423], [0.281, 0.525, 0.719, 0.118, -0.974, 0.808, 0.923],
                [0.246, -0.301, 0.231, 0.813, -0.702, 1.223, 1.105]])
rs = vector([0.096, 1.252, 1.024, 1.023, 1.155, 1.937, 1.673])
etalon = m.solve_right(rs)
excluded_rows = []
excluded_cols = []
print(m, rs)
for i in range(size):
    coords = get_max_abs(m, excluded_cols, excluded_rows)
    print(coords)
    excluded_cols.append(coords[1])
    excluded_rows.append(coords[0])
    
    m, rs = nullate_elements(m, rs, coords, excluded_rows)
    # print(m, rs)

vars = []
for i in range(size):
    vars.append(var(f'x{i}'))

print("Полученные урванения")
equations = []
for i in range(size):
    # equations.append(m[i][0]*vars[0]+m[i][1]*vars[1]+m[i][2]*vars[2]==rs[i])
    equations.append(sum([m[i][j]*vars[j] for j in range(size)])==rs[i])
print('\n'.join(map(str, equations)))
print("Порядок решения уравнений:", ', '.join(map(lambda x: str(x+1), list(reversed(excluded_rows)))))
fs = []
for i in range(len(list(reversed(excluded_rows)))):
    # print(len(equations), i, list(reversed(excluded_cols))[i])
    variable = vars[list(reversed(excluded_cols))[i]]
    print(equations[list(reversed(excluded_rows))[i]])
    solution = equations[list(reversed(excluded_rows))[i]].solve(variable, solution_dict=True)
    # print("sol", solution)
    solution = solution[0]
    print(solution)
    fs.append(solution)
    for j in range(i+1, len(list(reversed(excluded_rows)))):
        equations[list(reversed(excluded_rows))[j]] = equations[list(reversed(excluded_rows))[j]].subs(solution)
    # print(len(equations))

fs.sort(key=lambda x: str(list(x.keys())[0]))
print("Финальное решение:")
res = tuple(map(lambda x: RR(x[list(x.keys())[0]]), fs))
print(list(map(lambda x: abs(x[0]-x[1]), zip(res, etalon))))
print(etalon)

[ 411/1000  421/1000 -333/1000  313/1000 -141/1000 -381/1000    49/200]
[ 241/1000   141/200  139/1000 -409/1000  321/1000      1/16  101/1000]
[ 123/1000 -239/1000   251/500  901/1000  243/1000  819/1000  321/1000]
[ 413/1000  309/1000  801/1000   173/200  423/1000    59/500  183/1000]
[ 241/1000 -221/1000 -243/1000    67/500   637/500    89/125  423/1000]
[ 281/1000     21/40  719/1000    59/500  -487/500   101/125  923/1000]
[  123/500 -301/1000  231/1000  813/1000  -351/500 1223/1000   221/200] (0.0960000000000000, 1.25200000000000, 1.02400000000000, 1.02300000000000, 1.15500000000000, 1.93700000000000, 1.67300000000000)
(4, 4)
(6, 5)
(3, 2)
(5, 3)
(0, 1)
(2, 6)
(1, 0)
Полученные урванения
548575751247709/807745237558238*x0 + x1 + 571838751155737/807745237558238*x6 == 1.51327946505640
x0 == 11.0919696281674
3197812732669027/5757263014285571*x0 + x6 == 1.21133102986232
742391297/1829083785*x0 + 144626311/365816757*x1 + x2 + 1822320052/1829083785*x3 + 289413283/1829083785*x6 == 0.909